### 개체명 인식

* 도메인 또는 특정 목저(텍스트 마이닝)에 맞게 개체명 인식
* 과거
    - 구문분석 : 
        * 오늘 여러분은 점심 식사 이후에 12시 45분에 맞은편 커피숍인 빽다방 앞으로 오시면 커피를 드실 수 있습니다.
        * 1딘계 : 토크나이징
        * 2단계 : 품사를 태깅
        * 3단계 : 문법적 관계 파악: 주어 동사 목적어 보어
        * 4단계 : 주어에 있는 개체명 인식/ 목적에 있는 개체명 인식/ 보어에 있는 개체명 인식
        * 5단계 : 개체명으로 추정되는 데이터를 선택 예) '커피를'
        * 6단계 : (예: 가게명에 대한 개체명 인식) 뻭다방/ (예: 음식과 관련된 개체명 인식) 커피
        * 7단계 : 어떤 개체명인지 판단
        - 단점: 소요시간, 새로운 데이터가 들어왔을 때 인식이 어려움(유지보수가 필요)
* 딥러닝
    * 1단계: 훈련데이터셋 만들기
        * 1-1단계 : 토크나이징
        * 1-2 단계 :BIO 표현으로 라벨링 ( 아모레 퍼시픽 )
            - BIO: Begin(개체명의 시작되는 부분) (아모레), Inside(개체명의 내부부분) (퍼시픽), Outside(개체명이 아닌부분) (나머지)
    * 2단계: 모델 훈련하기
        - RNN, LSTM, GRU
    - 장점 : 소요시간 다소 줄었음 . 신규 데이터에 대한 개체명 인식 가능
            

In [50]:
import re
import urllib.request

In [51]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/12.%20RNN%20Sequence%20Labeling/dataset/train.txt",
                          filename="train.txt")

('train.txt', <http.client.HTTPMessage at 0x7f1fdfa6c520>)

In [52]:
f = open("train.txt", 'r')
tagged_sentences=[]
sentence= []

for line in f:
    if len(line)==0 or line.startswith('-DOCSTART') or line[0]=='\n':
        if len(sentence) >0:
            tagged_sentences.append(sentence)
            sentence = []
        continue
    splits = line.split(' ') #공백 기준으로 속성 분리
    splits[-1] = re.sub("\n","",splits[-1]) #BIO 태깅 부분 개행문자 제거
    word = splits[0].lower() #소문자로 변경
    sentence.append([word, splits[-1]])# 단어와 개체명 태깅만 저장
    

In [53]:
print(len(tagged_sentences))

14041


In [54]:
tagged_sentences[0]

[['eu', 'B-ORG'],
 ['rejects', 'O'],
 ['german', 'B-MISC'],
 ['call', 'O'],
 ['to', 'O'],
 ['boycott', 'O'],
 ['british', 'B-MISC'],
 ['lamb', 'O'],
 ['.', 'O']]

In [55]:
# 단어만 , 태깅만 데이터를 묶음
sentence, tag_info = zip(*tagged_sentences[0])
sentence, tag_info

(('eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'),
 ('B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'))

In [56]:
sentences, nar_tag = [], []
for tagged_sentence in tagged_sentences :
    sentence, tag_info = zip(*tagged_sentence)
    sentences.append(list(sentence))#단어정보
    nar_tag.append(list(tag_info)) #개체명 정보

In [57]:
sentences[0]

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']

### 개체명 인식을 위한 전처리

In [58]:
import re
import numpy as np
import matplotlib.pyplot as plt
import urllib.request

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences #문장 간 길이 조절
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [59]:
vacab_size = 4000

In [60]:
src_tokenizer = Tokenizer(num_words=vacab_size, oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(nar_tag)

In [61]:
tag_size= len(tar_tokenizer.word_index) +1
# 단어집합의 크기
print(vacab_size)
# 개체명 태깅 정보 집합 크기/라벨
print(tag_size)

4000
10


In [62]:
#정수 인코딩
X_train= src_tokenizer.texts_to_sequences(sentences)
y_train= tar_tokenizer.texts_to_sequences(nar_tag)

In [63]:
X_train[0], y_train[0]

([989, 1, 205, 629, 7, 3939, 216, 1, 3], [4, 1, 7, 1, 1, 1, 7, 1, 1])

In [64]:
# 원문 데이터
index_to_word= src_tokenizer.index_word
index_to_ner=tar_tokenizer.index_word

decoded = []

for index in X_train[0]:
    decoded.append(index_to_word[index])
print (f'기존 문장 : {sentences[0]}')   
print(f'빈도수가 낮은 문장은 OOV로 표현됨 :{decoded}')

기존 문장 : ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
빈도수가 낮은 문장은 OOV로 표현됨 :['eu', 'OOV', 'german', 'call', 'to', 'boycott', 'british', 'OOV', '.']


In [65]:
# 텍스트의 각 문장들의 길이가 다르기 떄문에 padding으로 모두 길이가 같게 조정
max_len=70
X_train = pad_sequences(X_train,padding='post', maxlen=max_len)
y_train = pad_sequences(y_train,padding='post',maxlen=max_len)

In [66]:
# 훈련 데이터 셋과 테스트 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.3, random_state=42)

In [67]:
# 원핫 인코딩
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [68]:
# 훈련 데이터 셋과 테스트 데이터 셋의 shape 확인
print('훈련 데이터 문장의 크기 : ', X_train.shape)
print('훈련 데이터 태깅/라벨의 크기 : ', y_train.shape)
print('테스트 데이터 문장의 크기 : ', X_test.shape)
print('테스트 데이터 태깅/라벨의 크기 : ', y_test.shape)

훈련 데이터 문장의 크기 :  (9828, 70)
훈련 데이터 태깅/라벨의 크기 :  (9828, 70, 10)
테스트 데이터 문장의 크기 :  (4213, 70)
테스트 데이터 태깅/라벨의 크기 :  (4213, 70, 10)


In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [70]:
from tensorflow import random as tf_ramdom, constant_initializer
import random
import numpy as np
#모델의 훈련 결과가 동일하게 나오도록 random_seed 값을 초기화
def reset_model_random():
    random_seed_num=0
    tf_ramdom.set_seed(random_seed_num)
    np.random.seed(random_seed_num)
    random.seed(random_seed_num)
    constant_initializer()

In [71]:
embedding_dim= 128
hidden_units= 128

In [74]:
model= Sequential()
model.add(Embedding(input_dim=vacab_size, output_dim=embedding_dim, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
#TimeDistributed() : LSTM을 다 vs 다로 구조로 출력

In [75]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 128)           512000    
                                                                 
 bidirectional (Bidirectiona  (None, 70, 256)          263168    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 70, 10)           2570      
 ibuted)                                                         
                                                                 
Total params: 777,738
Trainable params: 777,738
Non-trainable params: 0
_________________________________________________________________


In [78]:
reset_model_random()

es= EarlyStopping(patience=2, monitor='accuracy')
mc= ModelCheckpoint("NER_model.h5", save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

history = model.fit(X_train,y_train, batch_size=128,epochs=8, validation_data=(X_test,y_test)
                   ,callbacks=[es,mc])

Epoch 1/8


2023-05-12 04:23:29.968559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-05-12 04:23:31.362578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]


77/77 [==============================] - 13s 117ms/step - loss: 0.1076 - accuracy: 0.9680 - val_loss: 0.1641 - val_accuracy: 0.9550
Epoch 2/8
77/77 [==============================] - 8s 102ms/step - loss: 0.0936 - accuracy: 0.9725 - val_loss: 0.1652 - val_accuracy: 0.9556
Epoch 3/8
77/77 [==============================] - 8s 101ms/step - loss: 0.0854 - accuracy: 0.9744 - val_loss: 0.1654 - val_accuracy: 0.9564
Epoch 4/8
77/77 [==============================] - 8s 101ms/step - loss: 0.0759 - accuracy: 0.9772 - val_loss: 0.1675 - val_accuracy: 0.9551
Epoch 5/8
77/77 [==============================] - 8s 101ms/step - loss: 0.0671 - accuracy: 0.9801 - val_loss: 0.1705 - val_accuracy: 0.9541
Epoch 6/8
77/77 [==============================] - 8s 101ms/step - loss: 0.0607 - accuracy: 0.9821 - val_loss: 0.1746 - val_accuracy: 0.9565
Epoch 7/8
77/77 [==============================] - 8s 101ms/step - loss: 0.0548 - accuracy: 0.9839 - val_loss: 0.1797 - val_accuracy: 0.9540
Epoch 8/8
77/77 [=====

In [80]:
history.history

{'loss': [0.1076156497001648,
  0.09362948685884476,
  0.08539018034934998,
  0.07586663961410522,
  0.06705216318368912,
  0.06070725619792938,
  0.05480876937508583,
  0.04738142341375351],
 'accuracy': [0.96796053647995,
  0.9724546074867249,
  0.9744076132774353,
  0.9771796464920044,
  0.9800707101821899,
  0.9821427464485168,
  0.983934760093689,
  0.9862728118896484],
 'val_loss': [0.16407333314418793,
  0.1652088165283203,
  0.16536372900009155,
  0.1675347536802292,
  0.17047347128391266,
  0.17464490234851837,
  0.17965906858444214,
  0.18901048600673676],
 'val_accuracy': [0.9549534916877747,
  0.9556287527084351,
  0.956369936466217,
  0.9550852179527283,
  0.9541299343109131,
  0.9564687609672546,
  0.9539652466773987,
  0.9516593813896179]}

In [81]:
#모델 평가
model.evaluate (X_test, y_test)

132/132 [==============================] - 2s 14ms/step - loss: 0.1880 - accuracy: 0.9517


[0.18799300491809845, 0.9516593813896179]

In [82]:
#입력한 테스트용 샘플 데이터 y를 리턴
y_predicted = model.predict(np.array([X_test[0]]))


1/1 [==============================] - 1s 1s/step


In [83]:
y_predicted

array([[[3.64546476e-10, 9.99989867e-01, 1.36511085e-06, 1.75370246e-07,
         1.58650528e-06, 6.63754873e-09, 1.26299852e-07, 3.70170051e-06,
         8.78501396e-08, 2.94582037e-06],
        [1.02009807e-12, 9.99999762e-01, 1.26269608e-08, 5.37134504e-10,
         2.45928566e-09, 3.77602507e-12, 1.39514633e-09, 1.28637510e-07,
         3.31148331e-09, 1.01545133e-07],
        [1.30335271e-08, 9.96686637e-01, 5.90017939e-04, 2.93726858e-04,
         4.99723945e-04, 3.81677836e-08, 6.47259355e-07, 1.92341942e-03,
         4.28361773e-07, 5.39477651e-06],
        [8.80187005e-02, 1.10491298e-01, 1.08842015e-01, 9.89490077e-02,
         1.07820198e-01, 9.15997475e-02, 9.61341038e-02, 1.02968432e-01,
         9.61207896e-02, 9.90557447e-02],
        [8.80187005e-02, 1.10491298e-01, 1.08842015e-01, 9.89490077e-02,
         1.07820198e-01, 9.15997475e-02, 9.61341038e-02, 1.02968432e-01,
         9.61207896e-02, 9.90557447e-02],
        [8.80187005e-02, 1.10491298e-01, 1.08842015e-01, 9.8

In [84]:
#확률 벡터를 정수 레이블로 변경
y_predicted= np.argmax(y_predicted, axis=-1)

In [85]:
y_predicted

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]])

In [86]:
#원핫 백터를 정수로 인코딩
labels= np.argmax(y_test[0], axis=-1)
labels

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [92]:
print('{:15}| {:5} {}'.format('단어','실제값','예측값'))
print("="*40)
for word, tag, pred in zip(X_test[0], labels, y_predicted[0]):
    if word != 0: #을 제외/ PAD값
        print("{:17} : {:7} {}".format(index_to_word[word] , index_to_ner[tag] ,index_to_ner[pred]))

단어             | 실제값   예측값
attendance        : o       o
:                 : o       o
OOV               : o       o


In [97]:
#입력한 테스트용 샘플 데이터 y를 리턴
y_predicted = model.predict(np.array([X_test[90]]))
y_predicted= np.argmax(y_predicted, axis=-1)
labels= np.argmax(y_test[90], axis=-1)

print('{:15}| {:5} {}'.format('단어','실제값','예측값'))
print("-"*40)
for word, tag, pred in zip(X_test[90], labels, y_predicted[0]):
    if word != 0: #을 제외/ PAD값
        print("{:17} : {:7} {}".format(index_to_word[word] , index_to_ner[tag] ,index_to_ner[pred]))

1/1 [==============================] - 0s 23ms/step
단어             | 실제값   예측값
----------------------------------------
"                 : o       o
i                 : o       o
looked            : o       o
at                : o       o
it                : o       o
as                : o       o
not               : o       o
a                 : o       o
first             : o       o
round             : o       o
match             : o       o
,                 : o       o
just              : o       o
a                 : o       o
great             : o       o
challenge         : o       o
for               : o       o
me                : o       o
,                 : o       o
"                 : o       o
said              : o       o
coetzer           : b-per   b-per
,                 : o       o
24                : o       o
.                 : o       o
"                 : o       o
